In [15]:
import pandas as pd

In [5]:
checking = pd.read_csv("../data/Checking.csv", encoding='iso-8859-1')
checking.head()

,Date,nps,Comment
0,4/17/2023 20:23,9,PRECIO
1,4/17/2023 20:21,9,ES LA PRIMERA VEZ Y SI ME SIENTO CON ALGO DE I...
2,4/17/2023 20:17,9,Es muy rapido
3,4/17/2023 20:17,10,Bueno
4,4/17/2023 20:10,9,Prcticidad


In [9]:
checking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17860 entries, 0 to 17859
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Date     17860 non-null  object
 1   nps      17860 non-null  int64 
 2   Comment  17835 non-null  object
dtypes: int64(1), object(2)
memory usage: 418.7+ KB


In [13]:
sents = checking.Comment[30:40].to_list()
print(sents)

['es muy facil', 'ágil', 'rapidos', 'POR LA RAPIDEZ', 'super facil', 'ES MUY FACIL', 'si', 'rapida y eficiente la página', 'Muy eficiente', 'Es muy fácil y rápido.']


In [19]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization